In [1]:
import glob
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import random

# Personnal Import 
from utilities import DataSet, get_batch,get_mode_date2path
from load_data import load_subway_15_min

# Data
- Validation individuelles, aggrégée **3min**
- Metro (entrée/sortie)  **15 min**
- Sensor, aggrégé **6 min**

### Notes et questionnements
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

In [1]:
# Validation Individuelles :                        [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M, , , , , , , , , ],[J,F,M, , , , , , , , , ]
# Subway 15 Min :   Novembre 2019 - Mai 2021        [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]
# Sensor 6 Min :  Janvier 2019 - Decembre 2020      [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D]

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [5]:
folder_path = 'data/'
valid_ind_path = folder_path + 'Sub_Tram_11_2019_03_2020'
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
C:\Users\romai\AppData\Local\Temp\ipykernel_1708\2127861133.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[

## Load Subway 15 min

In [10]:
txt_path = "Métro 15 minutes 2019 2020.txt"

df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
df_metro_funi_2019_2020.head()

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0
3,2019-01-01 00:45:00,Ampère Victor Hugo,A,1,9.0
4,2019-01-01 01:00:00,Ampère Victor Hugo,A,0,0.0


## Load NetMob 15 min

## Load Sensor 6 min

# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.
